In [2]:
import pandas as pd
pd.set_option('display.max_columns', None)

import numpy as np

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler

from sklearn.pipeline import Pipeline, FunctionTransformer
from sklearn.compose import ColumnTransformer, make_column_selector

In [3]:
data = pd.read_csv('../raw_data/hospital_readmissions.csv')

# Function for Data-Cleaning

In [4]:
def make_clean_data(df: pd.DataFrame) -> pd.DataFrame:
    df = df[df['diag_1'] != 'Missing']
    df = df[df['diag_2'] != 'Missing']
    df = df[df['diag_3'] != 'Missing']

    df['age'] = df['age'].map({ '[0-10]': 0.0,
                                '[10-20)': 0.1,
                                '[20-30)': 0.2,
                                '[30-40)': 0.3,
                                '[40-50)': 0.4,
                                '[50-60)': 0.5,
                                '[60-70)': 0.6,
                                '[70-80)': 0.7,
                                '[80-90)': 0.8,
                                '[90-100)': 0.9,
                                '[100-110)': 1.0})
    df['n_lab_procedures_grouped'] = (df['n_lab_procedures'] // 10).astype(int)
    df['n_medications_grouped'] = (df['n_medications'] // 5).astype(int)
    df['n_outpatient'] = df['n_outpatient'].map({0: 0, 1: 1}).fillna(2).astype(int)
    df['n_inpatient'] = df['n_inpatient'].map({0: 0, 1: 1}).fillna(2).astype(int)
    df['n_emergency'] = df['n_emergency'].map({0: 0, 1: 1}).fillna(2).astype(int)

    df = df.drop(columns=['n_lab_procedures',
                          'medical_specialty',
                          'glucose_test',
                          'n_medications'],
                )
    return df

In [5]:
clean_data = make_clean_data(data)
clean_data.columns

Index(['age', 'time_in_hospital', 'n_procedures', 'n_outpatient',
       'n_inpatient', 'n_emergency', 'diag_1', 'diag_2', 'diag_3', 'A1Ctest',
       'change', 'diabetes_med', 'readmitted', 'n_lab_procedures_grouped',
       'n_medications_grouped'],
      dtype='object')

# Pre-Processing Blocks

In [6]:
data_cleaner = FunctionTransformer(make_clean_data)

In [7]:
num_preproc = Pipeline([
    ('scaler', MinMaxScaler()),
])

In [8]:
cat_preproc = Pipeline([
    ('ohe', OneHotEncoder(sparse_output=False, drop="if_binary")),
])

In [9]:
preproc = ColumnTransformer([
    ('num_transf', num_preproc, make_column_selector(dtype_include='number')),
    ('cat_transf', cat_preproc, make_column_selector(dtype_include='object')),
], verbose_feature_names_out=False).set_output(transform='pandas')

# Full Pre-Processing Pipeline

In [10]:
pipe_preproc = Pipeline([
    ('data_cleaner', data_cleaner),
    ('preprocessor', preproc),
])

pipe_preproc

Pipeline(steps=[('data_cleaner',
                 FunctionTransformer(func=<function make_clean_data at 0x15ef1cca0>)),
                ('preprocessor',
                 ColumnTransformer(transformers=[('num_transf',
                                                  Pipeline(steps=[('scaler',
                                                                   MinMaxScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x15ef37e80>),
                                                 ('cat_transf',
                                                  Pipeline(steps=[('ohe',
                                                                   OneHotEncoder(drop='if_binary',
                                                                                 sparse_output=False))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x15ef36fe0>)],
                                   verbose_feature_names_out=False))])

# check, that it works...

In [11]:
print(f'Preprocessing {data.shape[0]} rows of {data.shape[1]} features...')
preproc_data = pipe_preproc.fit_transform(data)
display(preproc_data)

Preprocessing 25000 rows of 17 features...


,age,time_in_hospital,n_procedures,n_outpatient,n_inpatient,n_emergency,n_lab_procedures_grouped,n_medications_grouped,diag_1_Circulatory,diag_1_Diabetes,diag_1_Digestive,diag_1_Injury,diag_1_Musculoskeletal,diag_1_Other,diag_1_Respiratory,diag_2_Circulatory,diag_2_Diabetes,diag_2_Digestive,diag_2_Injury,diag_2_Musculoskeletal,diag_2_Other,diag_2_Respiratory,diag_3_Circulatory,diag_3_Diabetes,diag_3_Digestive,diag_3_Injury,diag_3_Musculoskeletal,diag_3_Other,diag_3_Respiratory,A1Ctest_high,A1Ctest_no,A1Ctest_normal,change_yes,diabetes_med_yes,readmitted_yes
0,0.6,0.538462,0.166667,1.0,0.0,0.0,0.636364,0.200000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1,0.6,0.153846,0.333333,0.0,0.0,0.0,0.272727,0.133333,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,0.2,0.307692,0.000000,0.0,0.0,0.0,0.363636,0.200000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0
3,0.6,0.076923,0.000000,0.5,0.0,0.0,0.272727,0.133333,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0
4,0.4,0.000000,0.000000,0.0,0.0,0.0,0.363636,0.066667,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,0.8,1.000000,0.166667,0.0,0.0,0.0,0.636364,0.400000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
24996,0.8,0.076923,0.000000,0.0,0.0,0.0,0.545455,0.266667,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0
24997,0.6,0.307692,0.000000,0.0,0.5,0.0,0.090909,0.066667,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
24998,0.6,0.076923,0.500000,0.0,0.0,0.0,0.545455,0.200000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0


In [12]:
preproc_data.columns

Index(['age', 'time_in_hospital', 'n_procedures', 'n_outpatient',
       'n_inpatient', 'n_emergency', 'n_lab_procedures_grouped',
       'n_medications_grouped', 'diag_1_Circulatory', 'diag_1_Diabetes',
       'diag_1_Digestive', 'diag_1_Injury', 'diag_1_Musculoskeletal',
       'diag_1_Other', 'diag_1_Respiratory', 'diag_2_Circulatory',
       'diag_2_Diabetes', 'diag_2_Digestive', 'diag_2_Injury',
       'diag_2_Musculoskeletal', 'diag_2_Other', 'diag_2_Respiratory',
       'diag_3_Circulatory', 'diag_3_Diabetes', 'diag_3_Digestive',
       'diag_3_Injury', 'diag_3_Musculoskeletal', 'diag_3_Other',
       'diag_3_Respiratory', 'A1Ctest_high', 'A1Ctest_no', 'A1Ctest_normal',
       'change_yes', 'diabetes_med_yes', 'readmitted_yes'],
      dtype='object')